In [ ]:
%env kubectl=/Users/theganyo/dev/google-cloud-sdk/bin/kubectl
%env istio=/Users/theganyo/Downloads/istio-1.0.2
%env istioctl=/Users/theganyo/Downloads/istio-1.0.2/bin/istioctl
%env ingress=35.225.154.183
%env demo=/Users/theganyo/dev/fazio/microservices-demo

---
## Fazio Retail Shop UI

https://35.225.154.183/

---
## Architectural Diagram

![architecture](https://raw.githubusercontent.com/theganyo/microservices-demo/fazio-playground/img/architecture-diagram.jpeg)

---
## I've heard that blue is hip color!

But is it really? Let's canary test a blue front-end. We'll route 20% of traffic to the blue version.

In [ ]:
%%bash
cat <<EOF | $kubectl apply -f -

apiVersion: networking.istio.io/v1alpha3
kind: VirtualService
metadata:
  name: frontend-ingress
spec:
  gateways:
  - frontend-gateway
  hosts:
  - '*'
  http:
  - route:
    - destination:
        host: frontend
        subset: grey
      weight: 80
    - destination:
        host: frontend
        subset: blue
      weight: 20

EOF

---
## It turns out, customers prefer grey.

Let's revert that change.

In [ ]:
%%bash
cat <<EOF | $kubectl apply -f -

apiVersion: networking.istio.io/v1alpha3
kind: VirtualService
metadata:
  name: frontend-ingress
spec:
  gateways:
  - frontend-gateway
  hosts:
  - '*'
  http:
  - route:
    - destination:
        host: frontend
        subset: grey
      weight: 100

EOF

---
## Jaeger UI

Run in console:

    kubectl port-forward -n istio-system $(kubectl get pod -n istio-system -l app=jaeger -o jsonpath='{.items[0].metadata.name}') 16686:16686 &
    
http://localhost:16686/search?service=emailservice

---
## Grafana UI

Run in console:

    kubectl -n istio-system port-forward $(kubectl -n istio-system get pod -l app=grafana -o jsonpath='{.items[0].metadata.name}') 3000:3000 &
    
http://localhost:3000/d/3/pilot-dashboard

http://localhost:3000/d/LJ_uJAvmk/istio-service-dashboard?var-service=frontend.default.svc.cluster.local